### This notebook demonstrates how to make prediction uisng MHCRule.

Note, with unpickle (Jype JUnpickler especially), you may encounter error: 

<p style="color: blue">java.lang.NoSuchMethodError: java.nio.ByteBuffer.position(I)Ljava/nio/ByteBuffer;</p>

For this error, just uninstalled and then reinstalled jpype (conda install -c conda-forge jpype1) and it resolved



In [4]:
# import libraries
import pandas as pd
import numpy as np
import time
import itertools
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from rulekit.classification import RuleClassifier, ExpertRuleClassifier
from rulekit.params import Measures
from rulekit._helpers import *
from jpype.pickle import JPickler, JUnpickler

from sklearn.metrics import accuracy_score, f1_score, precision_recall_curve, roc_curve, auc
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV

from sklearn.model_selection import train_test_split
from src.utils import *
from src.utils_rules import *
from src.MHCRule import *

In [36]:
# initialize rulekit
_ = RuleClassifier()

In [27]:
# Load model
with open('./model/MHCRuleHydroPep.pkl','rb') as f:
    hydropep = pickle.load(f)
    
f.close()

## model

In [28]:
# supported alleles
print("Supported alleles: ", len(hydropep.alleles))
print(hydropep.alleles)

# Scale used for encoding
print("\nScale used for encoding peptide: ", hydropep.scale)

Supported alleles:  101
['HLA-A*02:01', 'HLA-A*03:01', 'HLA-A*11:01', 'HLA-A*02:03', 'HLA-A*31:01', 'HLA-A*02:06', 'HLA-A*68:02', 'HLA-B*07:02', 'HLA-A*01:01', 'HLA-B*15:01', 'HLA-A*26:01', 'HLA-A*68:01', 'HLA-A*02:02', 'HLA-A*24:02', 'HLA-B*58:01', 'HLA-B*27:05', 'HLA-A*33:01', 'HLA-B*35:01', 'HLA-B*08:01', 'HLA-B*40:01', 'HLA-A*30:01', 'HLA-B*57:01', 'HLA-B*51:01', 'HLA-B*18:01', 'HLA-A*29:02', 'HLA-A*69:01', 'HLA-A*23:01', 'HLA-A*30:02', 'HLA-B*44:02', 'HLA-B*46:01', 'HLA-B*53:01', 'HLA-B*39:01', 'HLA-B*44:03', 'HLA-B*15:17', 'HLA-A*02:19', 'HLA-A*24:03', 'HLA-B*54:01', 'HLA-A*02:12', 'HLA-A*80:01', 'HLA-A*32:01', 'HLA-A*02:11', 'HLA-B*40:02', 'HLA-B*45:01', 'HLA-B*08:02', 'HLA-A*25:01', 'HLA-A*02:16', 'HLA-B*27:03', 'HLA-B*48:01', 'HLA-B*15:09', 'HLA-B*15:03', 'HLA-A*26:02', 'HLA-A*26:03', 'HLA-B*38:01', 'HLA-C*04:01', 'HLA-B*08:03', 'HLA-C*07:01', 'HLA-C*06:02', 'HLA-A*02:17', 'HLA-B*83:01', 'HLA-C*03:04', 'HLA-B*35:03', 'HLA-C*14:02', 'HLA-C*05:01', 'HLA-B*14:02', 'HLA-B*42:01', 

In [13]:
# to get rules for an allele
allele = 'HLA-C*14:02'
hydropep.get_rules(allele)

['IF att2 = {Y} THEN label = {1}',
 'IF att8 = {A} THEN label = {1}',
 'IF att7 = {S} THEN label = {1}',
 'IF att1 = {Y} THEN label = {1}',
 'IF att8 = {P} THEN label = {1}',
 'IF att5 = {N} THEN label = {0}',
 'IF att1 = {R} THEN label = {0}',
 'IF att9 = {F} THEN label = {0}',
 'IF att9 = <0.11, inf) AND att5 = (-inf, 0.88) AND att1 = (-inf, 0.43) THEN label = {1}',
 'IF att6 = <0.21, inf) AND att1 = (-inf, 0.43) AND att3 = (-inf, 0.62) THEN label = {1}',
 'IF att2 = <0.11, 0.24) AND att4 = <0.43, inf) THEN label = {1}',
 'IF att8 = <0.11, inf) AND att7 = <0.43, inf) AND att2 = (-inf, 0.27) THEN label = {1}',
 'IF att6 = <0.11, inf) AND att5 = <0.16, inf) AND att7 = <0.16, inf) AND att2 = (-inf, 0.84) AND att1 = (-inf, 0.93) AND att3 = (-inf, 0.24) THEN label = {1}',
 'IF att5 = (-inf, 0.54) AND att8 = <0.24, 0.48) AND att7 = <0.033, inf) AND att2 = (-inf, 0.84) AND att1 = (-inf, 0.73) AND att3 = (-inf, 0.81) THEN label = {1}',
 'IF att9 = (-inf, 0.11) AND att7 = (-inf, 0.16) AND att

## predict using model

In [17]:
# test data
allele = 'HLA-A*02:01'
peptide = ['ALIRILQQL']
peptide = get_vector_representation(peptide)
peptide

array([['A', 'L', 'I', 'R', 'I', 'L', 'Q', 'Q', 'L']], dtype='<U1')

In [16]:
# predict label
## return_rules determines if rules are returned. 
## str_rules determines if str format rules are returned or coverage matrix is returned

hydropep.predict(allele, peptide, return_rules=True, str_rules=True)

(array([1]),
 [['IF att7 = {Q} THEN label = {0}',
   'IF att4 = {R} THEN label = {0}',
   'IF att2 = {L} THEN label = {1}',
   'IF att9 = {L} AND att6 = {L} THEN label = {1}',
   'IF att9 = {L} THEN label = {1}']],
 [['IF att5 = <0.11, 0.93) AND att7 = <0.48, inf) AND att1 = <0.24, inf) AND att3 = <0.11, inf) THEN label = {0}',
   'IF att5 = <0.033, 1.00) AND att7 = <0.27, inf) AND att4 = (-inf, 0.93) THEN label = {0}',
   'IF att9 = <0.11, 0.32) AND att7 = (-inf, 0.81) AND att2 = <0.11, 0.16) AND att1 = (-inf, 0.73) THEN label = {1}',
   'IF att9 = <0.11, 0.48) AND att6 = <0.033, 0.54) AND att7 = (-inf, 0.81) AND att2 = <0.11, 0.16) AND att1 = (-inf, 0.93) THEN label = {1}',
   'IF att9 = (-inf, 0.32) AND att7 = (-inf, 1.00) AND att2 = <0.11, 0.16) AND att1 = (-inf, 0.93) AND att3 = (-inf, 0.62) THEN label = {1}',
   'IF att9 = <0.11, 0.48) AND att6 = (-inf, 0.54) AND att7 = (-inf, 1.00) AND att2 = <0.11, 0.16) AND att1 = (-inf, 0.93) THEN label = {1}',
   'IF att9 = <0.11, 0.48) AND 

In [18]:
# predict binding probability
## return_rules determines if rules are returned. 
## str_rules determines if str format rules are returned or coverage matrix is returned
hydropep.predict_proba(allele, peptide, return_rules=True, str_rules=True)

(array([0.67716413]),
 [['IF att7 = {Q} THEN label = {0}',
   'IF att4 = {R} THEN label = {0}',
   'IF att2 = {L} THEN label = {1}',
   'IF att9 = {L} AND att6 = {L} THEN label = {1}',
   'IF att9 = {L} THEN label = {1}']],
 [['IF att5 = <0.11, 0.93) AND att7 = <0.48, inf) AND att1 = <0.24, inf) AND att3 = <0.11, inf) THEN label = {0}',
   'IF att5 = <0.033, 1.00) AND att7 = <0.27, inf) AND att4 = (-inf, 0.93) THEN label = {0}',
   'IF att9 = <0.11, 0.32) AND att7 = (-inf, 0.81) AND att2 = <0.11, 0.16) AND att1 = (-inf, 0.73) THEN label = {1}',
   'IF att9 = <0.11, 0.48) AND att6 = <0.033, 0.54) AND att7 = (-inf, 0.81) AND att2 = <0.11, 0.16) AND att1 = (-inf, 0.93) THEN label = {1}',
   'IF att9 = (-inf, 0.32) AND att7 = (-inf, 1.00) AND att2 = <0.11, 0.16) AND att1 = (-inf, 0.93) AND att3 = (-inf, 0.62) THEN label = {1}',
   'IF att9 = <0.11, 0.48) AND att6 = (-inf, 0.54) AND att7 = (-inf, 1.00) AND att2 = <0.11, 0.16) AND att1 = (-inf, 0.93) THEN label = {1}',
   'IF att9 = <0.11, 0

## Add new allele
If new allele data is available and needs to be included, it can be done in following manner.

We will use allele HLA-A*02:01 as an example. Since it is already a supported allele for 9, we will train for peptides with length 10

NOTES: 
1. In case rules for a specific allele needs to be retrained, pass retrain=True while fitting.

2. Finally, multiple alleles are to be trained use fit_loop() fn from MHCRule.py

In [25]:
# read data file
allele = 'HLA-A*02:01'
BA_df = pd.read_csv('./Data/Data_HLA.csv', index_col=0)
BA_df = BA_df[BA_df['peptide_length']==10].reset_index(drop=True)
BA_df = BA_df[BA_df['allele']==allele].reset_index(drop=True)

# split train, test
train_df, test_df = train_test_split(BA_df, test_size=0.3, stratify=BA_df['y'].to_list())

# Now, splitting test data into half to get test and validation sets
test_df, valid_df = train_test_split(test_df, test_size=0.5)

# get train
X_train, y_train = get_vector_representation(train_df['peptide']), train_df['y'].to_numpy()

X_train.shape, y_train.shape

((2388, 10), (2388,))

In [35]:
# Start the timer
start_time = time.time()

### first train mhcrulepeponly
hydropep.mhcrulepeponly.fit(allele=allele+'_len10', 
                            X=X_train, y=y_train, 
                            retrain=False)

### Next train mhcrulehydro
hydropep.mhcrulehydro.fit(allele=allele+'_len10', 
                            X=X_train, y=y_train, 
                            retrain=False)

### Finally fit train hydropep log reg
hydropep.fit(allele=allele+'_len10', 
             X=X_train, y=y_train, 
             retrain=False)

# End the timer
end_time = time.time()
elapsed_time = end_time - start_time

print("Total training time in seconds: ", elapsed_time)

# supported alleles
print("Supported alleles: ", len(hydropep.alleles))
print(hydropep.alleles)


Total training time in seconds:  4.92079758644104
Supported alleles:  102
['HLA-A*02:01', 'HLA-A*03:01', 'HLA-A*11:01', 'HLA-A*02:03', 'HLA-A*31:01', 'HLA-A*02:06', 'HLA-A*68:02', 'HLA-B*07:02', 'HLA-A*01:01', 'HLA-B*15:01', 'HLA-A*26:01', 'HLA-A*68:01', 'HLA-A*02:02', 'HLA-A*24:02', 'HLA-B*58:01', 'HLA-B*27:05', 'HLA-A*33:01', 'HLA-B*35:01', 'HLA-B*08:01', 'HLA-B*40:01', 'HLA-A*30:01', 'HLA-B*57:01', 'HLA-B*51:01', 'HLA-B*18:01', 'HLA-A*29:02', 'HLA-A*69:01', 'HLA-A*23:01', 'HLA-A*30:02', 'HLA-B*44:02', 'HLA-B*46:01', 'HLA-B*53:01', 'HLA-B*39:01', 'HLA-B*44:03', 'HLA-B*15:17', 'HLA-A*02:19', 'HLA-A*24:03', 'HLA-B*54:01', 'HLA-A*02:12', 'HLA-A*80:01', 'HLA-A*32:01', 'HLA-A*02:11', 'HLA-B*40:02', 'HLA-B*45:01', 'HLA-B*08:02', 'HLA-A*25:01', 'HLA-A*02:16', 'HLA-B*27:03', 'HLA-B*48:01', 'HLA-B*15:09', 'HLA-B*15:03', 'HLA-A*26:02', 'HLA-A*26:03', 'HLA-B*38:01', 'HLA-C*04:01', 'HLA-B*08:03', 'HLA-C*07:01', 'HLA-C*06:02', 'HLA-A*02:17', 'HLA-B*83:01', 'HLA-C*03:04', 'HLA-B*35:03', 'HLA-C*14:

As seen from supported alleles count and list, we successfully added a new allele "HLA-A*02:01_len10".

In [ ]:
# # Uncomment following lines to save model

# with open('./model/MHCRuleHydroPep.pkl','wb') as f:
#     pickle.dump(hydropep, f)
    
# f.close()